In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=180,
    return_messages=True,
)

In [36]:
# 예시 프롬프트들을 프롬프트 메모리에 넣는 과정

examples = [
    {
        "title": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "title": "Jaws",
        "answer": "🦈🏖️🚣‍♂️"
    },
    {
        "title": "The Shining",
        "answer": "👨‍👩‍👦‍👦🪓👻"
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {title}?"),
        ("ai", "{answer}"),
    ]   # in list
) # 학습

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
) # 생성

print(example_prompt)

for example in example_prompt.examples:
    memory.save_context(
        {"input": example["title"]},
        {"output": example["answer"]},
    )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You have to give me expression of movie with three imojis"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

examples=[{'title': 'The Godfather', 'answer': '👨\u200d👨\u200d👦🔫🍝'}, {'title': 'Jaws', 'answer': '🦈🏖️🚣\u200d♂️'}, {'title': 'The Shining', 'answer': '👨\u200d👩\u200d👦\u200d👦🪓👻'}] example_prompt=ChatPromptTemplate(input_variables=['answer', 'title'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['title'], template='What do you know about {title}?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer'], template='{answer}'))])
The human mentions "Jaws" and the AI responds with shark, beach, and rowboat emojis. 🦈🏖️🚣‍♂️ The human brings up "Top Gun" and the AI responds with airplane, sunglasses, and fire emojis. ✈️🕶️🔥 The AI cannot recall the movie they asked about last, but when reminded of "Jaws" and "Top Gun," it responds with the corresponding emojis. When prompted with "Top Gun," the AI recalls the movie and responds appropriately. The human mentions "Psycho" but the AI does not provide a response, instead responding with shower, knife

In [40]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

In [38]:
invoke_chain("Top Gun")
invoke_chain("Psycho")
# Why do not recorded in memory?

✈️🕶️🔥The human mentions "Jaws" and "Top Gun," prompting the AI to respond with corresponding emojis. When asked about the movie it inquired about last, the AI responds with shower, knife, and old woman emojis. 🚿🔪👵 The human mentions "Jaws" and the AI responds with shark, beach, and rowboat emojis. 🦈🏖️🚣‍♂️ The human mentions "Top Gun" and the AI responds with airplane, sunglasses, and fire emojis. ✈️🕶️🔥🚿🔪👵The human mentions "Jaws" and "Top Gun," prompting the AI to respond with corresponding emojis. When asked about the movie it inquired about last, the AI responds with shower, knife, and old woman emojis. 🚿🔪👵 The human mentions "Jaws" and the AI responds with shark, beach, and rowboat emojis. 🦈🏖️🚣‍♂️ The human mentions "Top Gun" and the AI responds with airplane, sunglasses, and fire emojis. ✈️🕶️🔥 The human mentions "Psycho" and the AI apologizes for not being able to recall the previous movie asked about.

In [39]:
invoke_chain("Tell me what is the movie you asked about last.")

I apologize, but I am unable to recall the previous movie asked about.The human mentions "Jaws" and "Top Gun," prompting the AI to respond with corresponding emojis. When asked about the movie it inquired about last, the AI responds with shower, knife, and old woman emojis. 🚿🔪👵 The human mentions "Jaws" and the AI responds with shark, beach, and rowboat emojis. 🦈🏖️🚣‍♂️ The human mentions "Top Gun" and the AI responds with airplane, sunglasses, and fire emojis. ✈️🕶️🔥 The human mentions "Psycho" and the AI apologizes for not being able to recall the previous movie asked about. When asked about the movie it inquired about last, the AI responds with shower, knife, and old woman emojis. 🚿🔪👵